# Version and Move/Promote Dialogflow CX Agent (Across or Within GCP Projects)

This notebook facilitates repeatable versioning and movement of DFCX agents across GCP projects, intended mainly for promotion from (e.g.) a dev environment to QA or similar.


Once all changes are made in the source agent, this notebook:
1. Saves timestamped versions of all flows
2. Deploys those new versions to a specified Dialogflow environment
3. Exports that environment to a GCS bucket temporarily
4. Restores from the GCS bucket to the specified target agent

# Config

High level information about the agents, projects, and configuration.

In [1]:
# What agent should be versioned and exported?
SOURCE_PROJECT_NAME = "source-project-name"
SOURCE_AGENT_NAME = "source-agent-name"

# What prefix is added to each version name before the timestamp?
VERSION_PREFIX = "1.0-QA-01_"
# What environment should the new versions be moved to?
DF_ENV_TO_EXPORT_NAME = "ENV"

# Where should the export be stored during the process?
GCS_BUCKET = "destination-bucket"
GCS_OBJECT_NAME ="1.0-QA-01"

# What agent should be restored/overwritten?
TARGET_PROJECT_NAME = "target-project-name"
TARGET_AGENT_NAME = "target-agent-name"

_DON'T FORGET TO RUN THE PREVIOUS CELL AFTER UPDATING VALUES_

# SCRAPI Installation & GCP Setup

Installs the SCRAPI library for scripting Dialogflow CX and sets up a connection to the needed projects and agents.

This one will take a minute or two.

In [5]:
import importlib

# tuples of (import name, install name)
packages = [
    ("dfcx_scrapi", "dfcx-scrapi")
]

# Install dependences
install = False
for package in packages:
    if not importlib.util.find_spec(package[0]):
        print(f"Installing {package[1]}...")
        install = True
        %pip install {package[1]} -U -q

# Restart kernel if needed
if install:
    import IPython
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

print("Done installing.")

['dfcx-scrapi>=1.11.1']
Done installing.


# Authentication

There are a few different methods for authenticating this notebook.


### Local Auth

To run this locally, make sure you have the Google Cloud SDK installed and Application Default Credentials configured. No additional authorization should be necessary.

### Auth in Google Colab

To authenticate as the current Google user in a Colab notebook, run this cell:

In [6]:
import sys

if "google.colab" in sys.modules:
    from google.colab.auth import authenticate_user
    authenticate_user()

### Alternate User Login

Use the below instead to manually authenticate with a different Google user (service account impersonation TK).
- Run it and follow the instructions to open the link, sign in with your *target* GCP account.
- After clicking through to allow access, it will give you a code which you should copy back to the field outputted by the cell below.
- You may see a warning but it's safe to ignore and won't mess anything up.

In [ ]:
# import os
# os.environ["PROJECT_ID"] = SOURCE_PROJECT_NAME

# # Interactive prompt to auth in browser
# !gcloud auth application-default login --no-launch-browser

# # Set active project to the `project_id` above
# !gcloud auth application-default set-quota-project $PROJECT_ID

# Imports and additional setup

Sets up a bunch of stuff to talk with the Dialogflow instance.

In [2]:
from datetime import datetime
from time import sleep
import pytz

from dfcx_scrapi.core.agents import Agents
from dfcx_scrapi.tools.copy_util import CopyUtil
from dfcx_scrapi.core.versions import Versions
from dfcx_scrapi.core.environments import Environments
from dfcx_scrapi.core.operations import Operations

from google.cloud.dialogflowcx_v3beta1.types.agent import Agent
from google.api_core.operation import Operation

## Initialize classes and grab agents

In [ ]:
# Instantiate the core classes
# Creds will be automatically picked up from the environment
# or else should be passed in with `creds_path`

agents_instance = Agents()
copy_util_instance = CopyUtil()
versions_instance = Versions()
operations_instance = Operations()
environments_instance = Environments()

In [ ]:
# Grab information about source and target agents

source_agent = agents_instance.get_agent_by_display_name(
    project_id=SOURCE_PROJECT_NAME,
    display_name=SOURCE_AGENT_NAME
)

if source_agent is None:
    raise ValueError(
        f"Agent '{SOURCE_AGENT_NAME}' does not exist in project '{SOURCE_PROJECT_NAME}."
    )

target_agent = agents_instance.get_agent_by_display_name(
    project_id=TARGET_PROJECT_NAME,
    display_name=TARGET_AGENT_NAME
)

if target_agent is None:
    raise ValueError(
        f"Agent '{TARGET_AGENT_NAME}' does not exist in project '{TARGET_PROJECT_NAME}."
    )

## Define functions for managing versions/environments

In [ ]:
def version_flows(
    agent: Agent,
    prefix: str = None,
    exclude_flows: list[str] = None,
    include_flows: list[str] = None
) -> list[Operation]:
    """Creates a new version of flows in a
    """

    ops = []
    flows_map = copy_util_instance.flows.get_flows_map(agent.name, reverse=True)

    # By default, use all flows
    iter_flows = flows_map.values()

    if include_flows is not None:
        iter_flows = (flows_map[f] for f in include_flows)
    elif exclude_flows is not None:
        iter_flows = (flows_map[f] for f in flows_map if f not in exclude_flows)

    for flow_id in iter_flows:

        lro = versions_instance.create_version(
            flow_id=flow_id,
            display_name=f"{prefix}{datetime.now(pytz.utc).isoformat('T', 'seconds')}"
            # TODO: Automated description of the flow diff
        )
        ops.append(lro)

    return ops


def move_to_environment(
        agent: Agent,
        target_environment_display_name: str,
        version_ops: list[Operation]
) -> list[Operation]:
    ops = []

    target_environment = environments_instance.get_environment_by_display_name(
        display_name=target_environment_display_name,
        agent_id=agent.name
    )

    iter_version_ids = (op.metadata.version for op in version_ops)
    for version_id in iter_version_ids:
        lro = environments_instance.deploy_flow_to_environment(
            environment_id=target_environment.name,
            flow_version=version_id
        )
        ops.append(lro)

    return ops

# Version flows, deploy to environment, export, and restore

## **_ONLY RUN ONCE!_**

This will create timestamped versions of all flows in the bot _each time it's run_. It might take a second for them all to complete just like creating versions manually.

After running this, the versions are assigned to the target environment in DF (e.g., "QA") and the environment is exported to a GCS buckets and restored over the target agent.

## Create flow versions

In [ ]:
# Create flow versions
version_ops = version_flows(
    agent=source_agent,
    prefix=VERSION_PREFIX
)
while not all(op.done() for op in version_ops):
    sleep(1)
print("Versioning - DONE")

Versioning - DONE


## Deploy new versions to target environment

In [ ]:
environment_ops = move_to_environment(
    agent=source_agent,
    target_environment_display_name=DF_ENV_TO_EXPORT_NAME,
    version_ops=version_ops
)
while any(not op.done() for op in environment_ops):
    sleep(1)
print("Deploying to Environment - DONE")

Deploying to Environment - DONE


# Export Source Agent and Restore to Target

In [ ]:
# Export target environment
export_op_id = agents_instance.export_agent(
    agent_id=source_agent.name,
    gcs_bucket_uri=f"gs://{GCS_BUCKET}/{GCS_OBJECT_NAME}",
    environment_display_name=DF_ENV_TO_EXPORT_NAME,
    include_bq_export_settings=False
)
while not operations_instance.get_lro(export_op_id).done:
    sleep(1)
print("Agent Export - DONE")

Agent Export - DONE


In [ ]:
# Restore to target agent
restore_op_id = agents_instance.restore_agent(
    agent_id=target_agent.name,
    gcs_bucket_uri=f"gs://{GCS_BUCKET}/{GCS_OBJECT_NAME}",
    restore_option=1
)
while not operations_instance.get_lro(restore_op_id).done:
    sleep(1)
print("Agent Restore - DONE")

Agent Restore - DONE
